In [17]:
model_config = {
    'task': 'Congestion',
    'save_path': './ResNet18/',
    'log_file': './ResNet18_log.txt',
    'pretrained': None,
    'max_iters': 1000,
    'plot_roc': False,
    'arg_file': None,
    'cpu': False,
    'dataroot': '/home/palaniappan-r/Repos/boosted-UNet/training_datasets/congestion_train_set/congestion',
    'ann_file_train': './files/train.csv',
    'ann_file_test': './files/test.csv',
    'dataset_type': 'CongestionDataset',
    'batch_size': 4,
    'aug_pipeline': ['Flip'],
    'model_type': 'Congestion_Prediction_Net',
    'in_channels': 3,
    'out_channels': 1,
    'lr': 0.0002,
    'weight_decay': 0.0001,
    'loss_type': 'MSELoss',
    'eval_metric': ['NRMS', 'SSIM'],
    'ann_file': './files/train.csv',
    'test_mode': False
}


In [18]:
import time
import numpy as np
import mmcv
from torch.utils.data import DataLoader
import datasets

class Flip:
    _directions = ['horizontal', 'vertical']

    def __init__(self, keys=['feature', 'label'], flip_ratio=0.5, direction='horizontal', **kwargs):
        if direction not in self._directions:
            raise ValueError(f'Direction {direction} is not supported. Currently supported directions are {self._directions}')
        self.keys = keys
        self.flip_ratio = flip_ratio
        self.direction = direction

    def __call__(self, results):
        if np.random.random() < self.flip_ratio:
            for key in self.keys:
                if isinstance(results[key], list):
                    for v in results[key]:
                        mmcv.imflip_(v, self.direction)
                else:
                    mmcv.imflip_(results[key], self.direction)
        return results

class Rotation:
    def __init__(self, keys=['feature', 'label'], axis=(0, 1), rotate_ratio=0.5, **kwargs):
        self.keys = keys
        self.axis = {k: axis for k in keys} if isinstance(axis, tuple) else axis
        self.rotate_ratio = rotate_ratio
        self.directions = [0, -1, -2, -3]

    def __call__(self, results):
        if np.random.random() < self.rotate_ratio:
            rotate_angle = self.directions[int(np.random.random() * 4)]
            for key in self.keys:
                if isinstance(results[key], list):
                    for v in results[key]:
                        v = np.ascontiguousarray(np.rot90(v, rotate_angle, axes=self.axis[key]))
                else:
                    results[key] = np.ascontiguousarray(np.rot90(results[key], rotate_angle, axes=self.axis[key]))
        return results


class IterLoader:
    def __init__(self, dataloader):
        self._dataloader = dataloader
        self.iter_loader = iter(self._dataloader)

    def __next__(self):
        try:
            data = next(self.iter_loader)
        except StopIteration:
            time.sleep(1)
            self.iter_loader = iter(self._dataloader)
            data = next(self.iter_loader)
        return data

    def __len__(self):
        return len(self._dataloader)

    def __iter__(self):
        return self


def build_dataset(train_options):
    aug_methods = {
        'Flip': Flip(),
        'Rotation': Rotation(**train_options)
    }

    if 'aug_pipeline' in train_options and not train_options['test_mode']:
        pipeline = [aug_methods[method] for method in train_options.pop('aug_pipeline')]
    else:
        pipeline = None

    dataset_cls = datasets.__dict__[train_options.pop('dataset_type')]
    dataset = dataset_cls(**train_options, pipeline=pipeline)

    if train_options['test_mode']:
        return DataLoader(dataset=dataset, num_workers=1, batch_size=1, shuffle=False)
    else:
        return IterLoader(DataLoader(
            dataset=dataset,
            num_workers=1,
            batch_size=train_options.pop('batch_size'),
            shuffle=True,
            drop_last=True,
            pin_memory=True
        ))


In [19]:
dataset = build_dataset(model_config)

In [20]:
import torch
import torch.nn as nn
import torchvision.models as models

def generation_init_weights(module):
    def init_func(m):
        classname = m.__class__.__name__
        if hasattr(m, 'weight') and ('Conv' in classname or 'Linear' in classname):
            if m.weight is not None:
                nn.init.normal_(m.weight, 0.0, 0.02)
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)
    module.apply(init_func)

def load_state_dict(module, state_dict, strict=False, logger=None):
    unexpected_keys = []
    all_missing_keys = []
    err_msg = []

    metadata = getattr(state_dict, '_metadata', None)
    state_dict = state_dict.copy()
    if metadata is not None:
        state_dict._metadata = metadata

    def load(module, prefix=''):
        local_metadata = {} if metadata is None else metadata.get(prefix[:-1], {})
        module._load_from_state_dict(state_dict, prefix, local_metadata, True,
                                     all_missing_keys, unexpected_keys, err_msg)
        for name, child in module._modules.items():
            if child is not None:
                load(child, prefix + name + '.')

    load(module)
    load = None

    missing_keys = [key for key in all_missing_keys if 'num_batches_tracked' not in key]

    if unexpected_keys:
        err_msg.append(f'unexpected key in source state_dict: {", ".join(unexpected_keys)}\n')
    if missing_keys:
        err_msg.append(f'missing keys in source state_dict: {", ".join(missing_keys)}\n')

    if err_msg:
        err_msg.insert(0, 'The model and loaded state dict do not match exactly\n')
        err_msg = '\n'.join(err_msg)
        if strict:
            raise RuntimeError(err_msg)
        elif logger is not None:
            logger.warning(err_msg)
        else:
            print(err_msg)
    return missing_keys

class PredictionNet(nn.Module):
    def __init__(self, in_channels=3, out_channels=1, **kwargs):
        super().__init__()

    def forward(self, x):
        pass

    def init_weights(self, pretrained=None, strict=True, **kwargs):
        if isinstance(pretrained, str):
            new_dict = torch.load(pretrained, map_location='cpu')['state_dict']
            load_state_dict(self, new_dict, strict=strict, logger=None)
        elif pretrained is None:
            generation_init_weights(self)
        else:
            raise TypeError("'pretrained' must be a str or None.")

class Congestion_Prediction_Net(PredictionNet):
    def __init__(self):
        super().__init__()
        self.resnet = models.resnet18(pretrained=True)
        num_ftrs = self.resnet.fc.in_features
        self.final_conv = nn.Conv2d(num_ftrs, 1, kernel_size=1)  # Output 1 channel
        self.upsample = nn.Upsample(size=(256, 256), mode='bilinear', align_corners=True)  # Upsample to 256x256
        self.sigmoid = nn.Sigmoid()  # Use sigmoid to ensure output is in [0, 1]

    def forward(self, x):
        x = self.resnet.conv1(x)
        x = self.resnet.bn1(x)
        x = self.resnet.relu(x)
        x = self.resnet.maxpool(x)

        x = self.resnet.layer1(x)
        x = self.resnet.layer2(x)
        x = self.resnet.layer3(x)
        x = self.resnet.layer4(x)

        x = self.resnet.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.final_conv(x.unsqueeze(2).unsqueeze(3)) 

        x = self.upsample(x) 
        x = self.sigmoid(x)  
        return x

    def init_weights(self, pretrained=None, strict=True, **kwargs):
        super().init_weights(pretrained, strict)



In [21]:
model = Congestion_Prediction_Net()
model.init_weights()
model = model.cuda()
print(model)

Congestion_Prediction_Net(
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affin

/home/palaniappan-r/anaconda3/envs/RoutingCNN/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/palaniappan-r/anaconda3/envs/RoutingCNN/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [22]:
# Copyright 2022 CircuitNet. All rights reserved.

import functools

import torch.nn as nn
import torch.nn.functional as F

__all__ = ['L1Loss', 'MSELoss']


def build_loss(opt):
    return globals()[opt.pop('loss_type')]()

def reduce_loss(loss, reduction):
    reduction_enum = F._Reduction.get_enum(reduction)
    if reduction_enum == 0:
        return loss
    if reduction_enum == 1:
        return loss.mean()

    return loss.sum()


def mask_reduce_loss(loss, weight=None, reduction='mean', sample_wise=False):
    if weight is not None:
        assert weight.dim() == loss.dim()
        assert weight.size(1) == 1 or weight.size(1) == loss.size(1)
        loss = loss * weight

    if weight is None or reduction == 'sum':
        loss = reduce_loss(loss, reduction)
    elif reduction == 'mean':
        if weight.size(1) == 1:
            weight = weight.expand_as(loss)
        eps = 1e-12

        if sample_wise:
            weight = weight.sum(dim=[1, 2, 3], keepdim=True)
            loss = (loss / (weight + eps)).sum() / weight.size(0)
        else:
            loss = loss.sum() / (weight.sum() + eps)

    return loss

def masked_loss(loss_func):
    @functools.wraps(loss_func)
    def wrapper(pred,
                target,
                weight=None,
                reduction='mean',
                sample_wise=False,
                **kwargs):
        loss = loss_func(pred, target, **kwargs)
        loss = mask_reduce_loss(loss, weight, reduction, sample_wise)
        return loss

    return wrapper

@masked_loss
def l1_loss(pred, target):
    return F.l1_loss(pred, target, reduction='none')


@masked_loss
def mse_loss(pred, target):
    return F.mse_loss(pred, target, reduction='none')

class L1Loss(nn.Module):
    def __init__(self, loss_weight=100.0, reduction='mean', sample_wise=False):
        super().__init__()

        self.loss_weight = loss_weight
        self.reduction = reduction
        self.sample_wise = sample_wise

    def forward(self, pred, target, weight=None, **kwargs):
        return self.loss_weight * l1_loss(
            pred,
            target,
            weight,
            reduction=self.reduction,
            sample_wise=self.sample_wise)



class MSELoss(nn.Module):
    def __init__(self, loss_weight=100.0, reduction='mean', sample_wise=False):
        super().__init__()
        self.loss_weight = loss_weight
        self.reduction = reduction
        self.sample_wise = sample_wise

    def forward(self, pred, target, weight=None, **kwargs):
        return self.loss_weight * mse_loss(
            pred,
            target,
            weight,
            reduction=self.reduction,
            sample_wise=self.sample_wise)


In [23]:
loss = build_loss(model_config)

In [24]:
import torch.optim as optim

optimizer = optim.AdamW(
    model.parameters(),
    lr=model_config['lr'],
    betas=(0.9, 0.999),
    weight_decay=model_config['weight_decay']
)


In [25]:
from math import cos, pi
from datetime import datetime
class CosineRestartLr(object):
    def __init__(self, base_lr, periods, restart_weights=[1], min_lr=None, min_lr_ratio=None):
        self.periods = periods
        self.min_lr = min_lr
        self.min_lr_ratio = min_lr_ratio
        self.restart_weights = restart_weights
        self.base_lr = base_lr

        self.cumulative_periods = [
            sum(self.periods[0:i + 1]) for i in range(len(self.periods))
        ]

    def annealing_cos(self, start: float, end: float, factor: float, weight: float = 1.) -> float:
        cos_out = cos(pi * factor) + 1
        return end + 0.5 * weight * (start - end) * cos_out

    def get_position_from_periods(self, iteration: int, cumulative_periods):
        for i, period in enumerate(cumulative_periods):
            if iteration < period:
                return i
        raise ValueError(f'Current iteration {iteration} exceeds cumulative_periods {cumulative_periods}')

    def get_lr(self, iter_num, base_lr: float):
        target_lr = self.min_lr if self.min_lr is not None else base_lr

        idx = self.get_position_from_periods(iter_num, self.cumulative_periods)
        current_weight = self.restart_weights[idx]
        nearest_restart = 0 if idx == 0 else self.cumulative_periods[idx - 1]
        current_periods = self.periods[idx]

        alpha = min((iter_num - nearest_restart) / current_periods, 1)
        return self.annealing_cos(base_lr, target_lr, alpha, current_weight)

    def _set_lr(self, optimizer, lr_groups):
        if isinstance(optimizer, dict):
            for k, optim in optimizer.items():
                for param_group, lr in zip(optim.param_groups, lr_groups[k]):
                    param_group['lr'] = lr
        else:
            for param_group, lr in zip(optimizer.param_groups, lr_groups):
                param_group['lr'] = lr

    def get_regular_lr(self, iter_num):
        return [self.get_lr(iter_num, _base_lr) for _base_lr in self.base_lr]

    def set_init_lr(self, optimizer):
        for group in optimizer.param_groups:
            group.setdefault('initial_lr', group['lr'])
            self.base_lr = [group['initial_lr'] for group in optimizer.param_groups]


In [26]:
cosine_lr = CosineRestartLr(model_config['lr'], [model_config['max_iters']], [1], 1e-7)
cosine_lr.set_init_lr(optimizer)

In [28]:
test_config = {
    'task': 'Congestion',
    'save_path': './Congestion/',
    'pretrained': None,
    'max_iters': 200000,
    'plot_roc': False,
    'arg_file': None,
    'cpu': False,
    'dataroot': '/home/palaniappan-r/Repos/boosted-UNet/training_datasets/congestion_train_set/congestion',
    'ann_file_train': './files/train.csv',
    'ann_file_test': './files/test.csv',
    'dataset_type': 'CongestionDataset',
    'batch_size': 4,
    'aug_pipeline': ['Flip'],
    'model_type': 'Congestion_Prediction_Net',
    'in_channels': 3,
    'out_channels': 1,
    'lr': 0.0002,
    'weight_decay': 0.0001,
    'loss_type': 'MSELoss',
    'eval_metric': ['NRMS', 'SSIM'],
    'ann_file': './files/test.csv',
    'test_mode': True
}


In [29]:
dataset = build_dataset(test_config)


In [30]:

# Copyright 2022 CircuitNet. All rights reserved.

from __future__ import print_function
import os
import os.path as osp
import numpy as np
from tqdm import tqdm
from utils.metrics import build_metric


def inference(test_config, model):
    # Build metrics
    metrics = {k:build_metric(k) for k in test_config['eval_metric']}
    avg_metrics = {k:0 for k in test_config['eval_metric']}

    count =0
    with tqdm(total=len(dataset)) as bar:
        for feature, label, label_path in dataset:
            if test_config['cpu']:
                input, target = feature, label
            else:
                input, target = feature.cuda(), label.cuda()

            prediction = model(input)
            for metric, metric_func in metrics.items():
                if not metric_func(target.cpu(), prediction.squeeze(1).cpu()) == 1:
                    avg_metrics[metric] += metric_func(target.cpu(), prediction.squeeze(1).cpu())

            if test_config['plot_roc']:
                save_path = osp.join(test_config['save_path'], 'test_result')
                if not os.path.exists(save_path):
                    os.makedirs(save_path)
                file_name = osp.splitext(osp.basename(label_path[0]))[0]
                save_path = osp.join(save_path, f'{file_name}.npy')
                output_final = prediction.float().detach().cpu().numpy()
                np.save(save_path, output_final)
                count +=1

            bar.update(1)


    for metric, avg_metric in avg_metrics.items():
        print("===> Avg. {}: {:.4f}".format(metric, avg_metric / len(dataset))) 

In [31]:
# Copyright 2022 CircuitNet. All rights reserved.

from __future__ import print_function
import os
import os.path as osp
import numpy as np
from tqdm import tqdm
from utils.metrics import build_metric


def inference(test_config, model):
    # Build metrics
    metrics = {k:build_metric(k) for k in test_config['eval_metric']}
    avg_metrics = {k:0 for k in test_config['eval_metric']}

    count =0
    with tqdm(total=len(dataset)) as bar:
        for feature, label, label_path in dataset:
            if test_config['cpu']:
                input, target = feature, label
            else:
                input, target = feature.cuda(), label.cuda()

            prediction = model(input)
            for metric, metric_func in metrics.items():
                if not metric_func(target.cpu(), prediction.squeeze(1).cpu()) == 1:
                    avg_metrics[metric] += metric_func(target.cpu(), prediction.squeeze(1).cpu())

            if test_config['plot_roc']:
                save_path = osp.join(test_config['save_path'], 'test_result')
                if not os.path.exists(save_path):
                    os.makedirs(save_path)
                file_name = osp.splitext(osp.basename(label_path[0]))[0]
                save_path = osp.join(save_path, f'{file_name}.npy')
                output_final = prediction.float().detach().cpu().numpy()
                np.save(save_path, output_final)
                count +=1

            bar.update(1)


    for metric, avg_metric in avg_metrics.items():
        print("===> Avg. {}: {:.4f}".format(metric, avg_metric / len(dataset))) 
    

In [32]:
inference(test_config, model)

100%|██████████| 3164/3164 [00:55<00:00, 56.65it/s]

===> Avg. NRMS: 0.0586
===> Avg. SSIM: 0.7676
